# Virtually Try on:


1. Install some Dependecies.
2. Clone the open pose library and build it.
3. Import all needed libraries.
4. Clone our project VITON from github .
5. Implement the virtual try on test function to use it in Dash after that
6. Create the Dashboard.
7. Run the dash and see the results





## 1. Install some Dependecies

In [1]:
!pip install ninja
!pip install opencv-python torchgeometry

# needed libraries for Dash
!pip install dash
!pip install jupyter-dash
!pip install dash-bootstrap-components

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 108 kB 14.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 1.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.9 MB 13.1 MB/s 
     |████████████████████████████████| 357 kB 48.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 16.9 MB/s 
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=42fdcf348de930d170cd9f297979eb7cbb5daf6b770253c14a44fbd94fc0ba65
  Stored in directory: /root/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying
Looking in indexes: https:/

## 2.Clone the open pose library and build it

In [2]:
%cd /content/
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`
  

/content
Selecting previously unselected package libgflags2.2.
(Reading database ... 155676 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.1-1_amd64.deb ...
Unpacking libgflags2.2 (2.2.1-1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.1-1_amd64.deb ...
Unpacking libgflags-dev (2.2.1-1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.3.5-1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to unpack .../03-libgoogle-glog-dev_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog-dev (0.3.5-1) ...
Selecting previously unselected package libhdf5-serial-dev.
Preparing to unpack .../04-libhdf5-serial-dev_1.10.0-patch1+docs-4_all.deb ...
Unpacking libhdf5-serial-dev (1.10.0-patch1+docs-4) ...
Selecting previously unselected package libleveldb1v5:amd64.
Preparing to u

## 3.Import some needed libraries

In [3]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import gdown
import requests
import cv2

## 4.Clone our project Virtual try on from github

In [4]:
!git clone https://github.com/NadaAdelMousa/Virtual-Try-on.git

Cloning into 'Virtual-Try-on'...
remote: Enumerating objects: 738, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 738 (delta 0), reused 7 (delta 0), pack-reused 728
Receiving objects: 100% (738/738), 54.12 MiB | 13.21 MiB/s, done.
Resolving deltas: 100% (168/168), done.


> ### Get checkpoints of the VITON model

In [5]:
# checkpoints alias 

url = 'https://drive.google.com/uc?id=1cAUuGnK1jY_69YQuaqIJ2EJLL5bLYKVN'
output = '/content/Virtual-Try-on/checkpoints/alias_final.pth'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1cAUuGnK1jY_69YQuaqIJ2EJLL5bLYKVN
To: /content/Virtual-Try-on/checkpoints/alias_final.pth
100%|██████████| 402M/402M [00:04<00:00, 95.0MB/s]


'/content/Virtual-Try-on/checkpoints/alias_final.pth'

In [6]:
# checkpoints gmm 

url = 'https://drive.google.com/uc?id=1W8teZFz_I4wZCasTQIsGa6el1HB4Vw8b'
output = '/content/Virtual-Try-on/checkpoints/gmm_final.pth'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1W8teZFz_I4wZCasTQIsGa6el1HB4Vw8b
To: /content/Virtual-Try-on/checkpoints/gmm_final.pth
100%|██████████| 76.2M/76.2M [00:00<00:00, 160MB/s]


'/content/Virtual-Try-on/checkpoints/gmm_final.pth'

In [7]:
# checkpoints seg 

url = 'https://drive.google.com/uc?id=1Nai8ladjicPGrXGHDFxrGo-u2iSsAWSV'
output = '/content/Virtual-Try-on/checkpoints/seg_final.pth'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1Nai8ladjicPGrXGHDFxrGo-u2iSsAWSV
To: /content/Virtual-Try-on/checkpoints/seg_final.pth
100%|██████████| 138M/138M [00:01<00:00, 107MB/s]


'/content/Virtual-Try-on/checkpoints/seg_final.pth'

> ### get the segmentation checkpoints

In [8]:
%cd /content/Virtual-Try-on/Self-Correction-Human-Parsing
!mkdir checkpoints

/content/Virtual-Try-on/Self-Correction-Human-Parsing


In [9]:
dataset = 'lip'         #select from ['lip', 'atr', 'pascal']

In [10]:
if dataset == 'lip':
    url = 'https://drive.google.com/uc?id=1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH'
elif dataset == 'atr':
    url = 'https://drive.google.com/uc?id=1ruJg4lqR_jgQPj-9K0PP-L2vJERYOxLP'
elif dataset == 'pascal':
    url = 'https://drive.google.com/uc?id=1E5YwNKW2VOEayK9mWCS3Kpsxf-3z04ZE'

output = 'checkpoints/final.pth'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH
To: /content/Virtual-Try-on/Self-Correction-Human-Parsing/checkpoints/final.pth
100%|██████████| 267M/267M [00:04<00:00, 66.4MB/s]


'checkpoints/final.pth'

> ### create a folder for svaing test images and outputs 

In [11]:
%cd /content/Virtual-Try-on/datasets
with open('test_pairs.txt', 'w') as f:
    f.write('person1.jpg cloth1.jpg')

/content/Virtual-Try-on/datasets


In [12]:
!mkdir /content/Virtual-Try-on/datasets/test
%cd /content/Virtual-Try-on/datasets/test

!mkdir cloth
!mkdir cloth-mask
!mkdir image
!mkdir image-parse
!mkdir openpose-img
!mkdir openpose-json

/content/Virtual-Try-on/datasets/test


## 5.Implement the virtual try on test function to use it in Dash after that

In [13]:
def viton():

  person_img_path = '/content/Virtual-Try-on/datasets/test/image/person1.jpg'
  cloth_img_path = '/content/Virtual-Try-on/datasets/test/cloth/cloth1.jpg'

  img = Image.open('/content/img1.jpg').resize((768, 1024))
  img.save(person_img_path)

  img = Image.open('/content/img2.jpg').resize((768, 1024))
  img.save(cloth_img_path)

  # segmentation part
  %cd /content/Virtual-Try-on/Self-Correction-Human-Parsing
  !python3 simple_extractor.py \
         --dataset 'lip' \
         --model-restore 'checkpoints/final.pth' \
         --input-dir '/content/Virtual-Try-on/datasets/test/image'  \
         --output-dir '/content/Virtual-Try-on/datasets/test/image-parse'


  # mask part 
  
  # im_parse = Image.open(cloth_img_path).convert('L') 
  # cloth_array = np.array(im_parse) 
  # cloth_mask = (cloth_array < 240)

  # im = Image.fromarray(cloth_mask)
  # im.save('/content/VITON-HD/datasets/test/cloth-mask/cloth1.jpg')

  img = cv2.imread(cloth_img_path, 0)
  blur = cv2.GaussianBlur(img,(5,5),0)
  ret3,th3 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
  cloth_mask = (th3 < 150)
  
  im = Image.fromarray(cloth_mask)
  im.save('/content/Virtual-Try-on/datasets/test/cloth-mask/cloth1.jpg')

  # open pose part
  !cd /content/openpose && ./build/examples/openpose/openpose.bin \
            --image_dir '/content/Virtual-Try-on/datasets/test/image'  \
            --write_json '/content/Virtual-Try-on/datasets/test/openpose-json'  \
            --display 0 --write_images '/content/Virtual-Try-on/datasets/test/openpose-img'  \
            --disable_blending --hand --render_pose 1 


  # run model
  %cd /content/Virtual-Try-on
  !python /content/Virtual-Try-on/test.py --name test

----------------------------------------------------------------------------------------------------------------------------

# 5. Create the Dashboard

> ## Import Libraries

In [14]:
import pandas as pd
import numpy as np
import plotly.express as px
import datetime
from PIL import Image

import dash
from dash import html
from dash import dcc
from jupyter_dash import JupyterDash

#from dash import Input, Output, State
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
from dash_bootstrap_components._components.Container import Container

from base64 import decodebytes
import base64


> ## Header of Dashboard

In [15]:
pages_nav = dbc.Nav(
            [
                dbc.NavLink("Home", href="/", active="exact", style= {"margin-right": "2rem"}),
                dbc.NavLink("D_Room", href="/D_Room", active="exact", style= {"margin-right": "2rem"}),
                dbc.NavLink("About_US", href="/About_US", active="exact", style= {"margin-right": "2rem"})
            ],
            #vertical=True,
            pills=True,
            className="g-0 ms-auto",
            #align="center",
    
        )

In [16]:
LOGO = "https://img.freepik.com/premium-vector/woman-trying-clothes-web-application-illustration-female-character-chooses-red-dress-hat-from-online-store-virtually-dresses-them-virtual-fitting-room_241107-892.jpg?w=2000"

header_navbar = dbc.Navbar(
    dbc.Container(
        [
            html.A(
                # Use row and col to control vertical alignment of logo / brand
                dbc.Row(
                    [
                        dbc.Col(html.Img(src=LOGO, height="30px")),
                        dbc.Col(dbc.NavbarBrand("Estilo", className="ms-3", style={'font-size':'1.5rem'})),
                    ],
                    align="center",
                    className="g-0",
                ),
                href="https://plotly.com",
                style={"textDecoration": "none"},
            ),
            dbc.NavbarToggler(id="navbar-toggler3", n_clicks=0),
            dbc.Collapse(
                pages_nav,
                id="navbar-collapse3",
                is_open=False,
                navbar=True,
            ),
        ]
    ),
    color= "rgb(43 42 62)",
    dark=True,
)

> ## Page 1 : Content --> Home

In [17]:
cover_img = 'https://img.freepik.com/premium-vector/woman-trying-clothes-web-application-illustration-female-character-chooses-red-dress-hat-from-online-store-virtually-dresses-them-virtual-fitting-room_241107-892.jpg?w=2000'

Home_page = html.Div(
    [
        dbc.Row(
        [
          dbc.Col(html.Div([html.H1("VIRTUAL", style={'font-size':'70px', 'color':'rgb(43, 42, 62)', 'text-align':'center', 'padding-top':'25%'}),
                            
                            html.P('Dressing Room', style={'font-size':'35px', 'color':'rgb(43, 42, 62)', 'text-align':'center'}),

                            html.P('A virtual dressing room website for most comfortable, convenient and time-saving online-shopping and virtual trying on.\
                                    The app is based on an AI solution. The technology provides users with a unique outstanding experience -\
                                    to try clothes online' , style={'font-size':'14px', 'text-align':'center'})
                           ],
                           #style={'padding': '20% 20%'}
                           ),
                  
                   width={'size':4, 'offset':1}
                  ),
                  
          dbc.Col(html.Img(src=cover_img, height='100%', width='800px'), width={"size": 3, "offset": 0}),
        ],
        #align="center",
          ),

    ])

> ## Page 2 : Content --> Dressing Room

 >> ### upload images of person and cloth

In [18]:
upload_img_person = html.Div([
        dcc.Upload(
            id='upload_image_person',
            children=html.Div([
                'Upload an image for a person',
                html.A('')
        ], style={'align': 'center'}),

        style={
            'width': '80%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'solid',
            'borderRadius': '10px',
            'textAlign': 'center',
            'margin': '10px auto',
            'align' :'center',
            'justify':'center',
            'background-color':'#dcdcdc',
        },

        style_active={
            'borderColor': 'red',
            'backgroundColor': '#eee'
        }
    ),
  
])


upload_img_cloth = html.Div([
        dcc.Upload(
            id='upload_image_cloth',
            children=html.Div([
                'Upload an image for cloth',
                html.A('')
        ]),
        style={
            'width': '80%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'solid',
            'borderRadius': '10px',
            'textAlign': 'center',
            'margin': '10px auto',
            'align' :'center',
            'background-color':'#dcdcdc'
        },
    ),
])


def parse_person(contents):
    return html.Div([
        # that is supplied by the upload
        html.Img(src=contents, style={'height': '100%', 'width':'100%'}),

    ])



def parse_close(contents):
    return html.Div([       
        # that is supplied by the upload
        html.Img(src=contents, style={'height': '100%', 'width':'100%'}),
    ])



>> ### Procees button to get the output image

In [19]:
# Procees butoon to get the output image
process_button= html.Div(
    [
        dbc.Button("Process", id='process_button', color="secondary", n_clicks=0, className="me-1",
                   style={
                      'width': '80%',
                      'height': '60px',
                      'font-size':'1.4rem',
                      'borderRadius': '10px',
                      'textAlign': 'center',
                      'margin': '10px auto',
                      'margin-left': '20px',
                      'align' :'center',
                      #'background-color':'#dcdcdc'
                   }),
        #html.Span(id="example-output", style={"verticalAlign": "middle"}),
    ],
    style={
        'margin':'0px auto',
        'align':'center'
    })

>> ### Output images of person , Coth and result of both

In [20]:
show_person = html.Div(id='output_person')
show_close = html.Div(id='output_close')

show_vton = html.Div(id='output_vton')

>> ### set the layout of dreassing room page

In [21]:
D_Room_page =  html.Div([
                       dbc.Row(
                       [
                        dbc.Col(upload_img_person, width={"size": 3, "offset": 0}, align="center"),
                        dbc.Col(upload_img_cloth, width={"size": 3, "offset": 0}, align="center"),
                        dbc.Col(process_button, width={"size": 3, "offset": 0}, align="center"),
                       ],
                      align="center",
                      justify="evenly",
                    
                        ),
                        
                        
                       dbc.Row(
                       [
                        dbc.Col(show_person, width={"size": 3, "offset": 0}, align="center", style={'height':'300px'}),
                        dbc.Col(show_close, width={"size": 3, "offset": 0}, align="center", style={'height':'300px'}),
                        dbc.Col(show_vton, width={"size": 3, "offset": 0}, align="center", style={'height':'300px'})
                       ],
                      align="center",
                      justify="evenly",
                    
                        )
                    ],
                    style={'padding':'20px 0px 40px 0px'})

> ## Page 3 : content --> Abou_us

In [22]:
About_US_page = html.Div(
    [
        
    ]
)

> ## The app

In [23]:
# create the app 
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

#app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

content = html.Div(id="page-content", children=[])


app.layout = html.Div([
    dcc.Location(id="url"),
    header_navbar,
    content
])



# image upload person
@app.callback(Output('output_person', 'children'),
              Input('upload_image_person', 'contents'))
              #State('upload-image', 'filename'),
              #State('upload-image', 'last_modified'))

def update_output_person(list_of_contents):

    if list_of_contents is not None:
        image = list_of_contents.split(',')[1]
        data = decodebytes(image.encode('ascii'))
        with open(f"/content/img1.jpg", "wb") as f:
            f.write(data)

        children = parse_person(list_of_contents)
        return children
    
    
# image upload close
@app.callback(Output('output_close', 'children'),
              Input('upload_image_cloth', 'contents'))
              #State('upload-image', 'filename'),
              #State('upload-image', 'last_modified'))

def update_output_cloth(list_of_contents):
    if list_of_contents is not None:
        
        image = list_of_contents.split(',')[1]
        data = decodebytes(image.encode('ascii'))
        with open(f"/content/img2.jpg", "wb") as f:
            f.write(data)

        children = parse_close(list_of_contents)
        return children
     

# process button 
@app.callback(
    Output("output_vton", 'children'),
    #Output("example-output", "children"),
    [Input("process_button", "n_clicks")]
)

def on_button_click(n):
    if n is not None:
        viton()
        image_vton = '/content/Virtual-Try-on/results/test/person1.jpg_cloth1.jpg'

        encoded_image = base64.b64encode(open(image_vton, 'rb').read())

        return html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height': '100%', 'width':'100%'}) 


# def on_button_click(n):
#     if n is None:
#         return 
#     else:
#         vton()
#         image_vton = '/content/VITON-HD/results/testiii/person1.jpg_cloth1.jpg'
#         return html.Img(src=image_vton, style={'height': '100%', 'width':'100%'})


# add callback for toggling the collapse on small screens
@app.callback(
    Output("navbar-collapse", "is_open"),
    [Input("navbar-toggler", "n_clicks")],
    [State("navbar-collapse", "is_open")],
)

def toggle_navbar_collapse(n, is_open):
    if n:
        return not is_open
    return is_open



@app.callback(
    Output("page-content", "children"),
    [Input("url", "pathname")]
)
def render_page_content(pathname):
    if pathname == "/":
        return Home_page
    
    elif pathname == "/D_Room":
        return D_Room_page

    elif pathname == "/About_US":
        return About_US_page
   
    # If the user tries to reach a different page, return a 404 message
    return dbc.Jumbotron(
        [
            html.H1("404: Not found", className="text-danger"),
            html.Hr(),
            html.P(f"The pathname {pathname} was not recognised..."),
        ]
    )

  

## 7.Run the dash and see the results

In [24]:
if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on:


<IPython.core.display.Javascript object>